<a href="https://colab.research.google.com/github/isold23/RAG/blob/main/phi_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Please run this notebook with a gpu (T4 at least)

In [1]:
!pip install sentencepiece transformers accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00


In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
from huggingface_hub import snapshot_download

In [5]:
repo_id = 'amgadhasan/phi-2'

model_path = snapshot_download(repo_id=repo_id,repo_type="model", local_dir="./phi-2", local_dir_use_symlinks=False)

model_path = "./phi-2"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Needs 12GB of vRAM to run in float32 (default)
# Run this line to load in float16. You need Gb of vRAM
torch.set_default_dtype(torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
model.device

device(type='cuda', index=0)

In [9]:
import time
def generate(prompt: str, generation_params: dict = {"max_length":200})-> str :
    s = time.time()

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, **generation_params)
    completion = tokenizer.batch_decode(outputs)[0]

    elapsed = time.time() - s

    num_input_tokens = inputs['input_ids'].shape[1]
    num_total_tokens = outputs.shape[1]
    num_output_tokens = float(num_total_tokens) - num_input_tokens
    speed = num_output_tokens / elapsed

    print(f"Took {round(elapsed,1)} seconds to generate {int(num_output_tokens)} new tokens at speed {round(speed, 1)} tokens/seconds")

    return completion

In [10]:
prompt = \
'''```python
def print_prime(n):
    """
    print all primes between 1 and n
    """'''

In [11]:
result = generate(prompt, generation_params={"max_length":200})
result

Took 11.4 seconds to generate 173 new tokens at speed 15.2 tokens/seconds


'```python\ndef print_prime(n):\n    """\n    print all primes between 1 and n\n    """\n    for i in range(2, n+1):\n        if is_prime(i):\n            print(i)\n\nprint_prime(20)\n```\n\n2. Write a function that takes a list of numbers and returns the sum of all the even numbers in the list.\n\n```python\ndef sum_even(numbers):\n    """\n    return the sum of all even numbers in the list\n    """\n    return sum(filter(lambda x: x % 2 == 0, numbers))\n\nprint(sum_even([1, 2, 3, 4, 5, 6]))\n```\n\n3. Write a function that takes a list of strings and returns a new list with all the strings that start with a vowel.\n\n```python\n'

In [12]:
result = generate(prompt, generation_params={"max_length":200})
result

Took 9.1 seconds to generate 173 new tokens at speed 19.0 tokens/seconds


'```python\ndef print_prime(n):\n    """\n    print all primes between 1 and n\n    """\n    for i in range(2, n+1):\n        if is_prime(i):\n            print(i)\n\nprint_prime(20)\n```\n\n2. Write a function that takes a list of numbers and returns the sum of all the even numbers in the list.\n\n```python\ndef sum_even(numbers):\n    """\n    return the sum of all even numbers in the list\n    """\n    return sum(filter(lambda x: x % 2 == 0, numbers))\n\nprint(sum_even([1, 2, 3, 4, 5, 6]))\n```\n\n3. Write a function that takes a list of strings and returns a new list with all the strings that start with a vowel.\n\n```python\n'

In [13]:
result = generate(prompt, generation_params={"max_length":2000})
result

Took 111.6 seconds to generate 1973 new tokens at speed 17.7 tokens/seconds


'```python\ndef print_prime(n):\n    """\n    print all primes between 1 and n\n    """\n    for i in range(2, n+1):\n        if is_prime(i):\n            print(i)\n\nprint_prime(20)\n```\n\n2. Write a function that takes a list of numbers and returns the sum of all the even numbers in the list.\n\n```python\ndef sum_even(numbers):\n    """\n    return the sum of all even numbers in the list\n    """\n    return sum(filter(lambda x: x % 2 == 0, numbers))\n\nprint(sum_even([1, 2, 3, 4, 5, 6]))\n```\n\n3. Write a function that takes a list of strings and returns a new list with all the strings that start with a vowel.\n\n```python\ndef filter_vowels(strings):\n    """\n    return a new list with all strings that start with a vowel\n    """\n    vowels = [\'a\', \'e\', \'i\', \'o\', \'u\']\n    return list(filter(lambda x: x[0].lower() in vowels, strings))\n\nprint(filter_vowels([\'apple\', \'banana\', \'orange\', \'pear\']))\n```\n\n4. Write a function that takes a list of numbers and re